전이학습을 이용한 한국어 법률 분야의 감정 분석과 문장 유사도 분석을 위한 파인튜닝을 추가로 설명하겠습니다. 이 코드 예제는 사전 학습된 한국어 모델을 사용하여 감정 분석 및 문장 유사도 측정을 수행
1. 전이학습 개요
전이학습은 주로 사전 학습된 모델을 사용하여 특정 태스크에 맞게 재학습하는 과정을 포함합니다. 예를 들어, 감정 분석 태스크에 적합한 사전 학습된 언어 모델을 선택한 후, 법률 분야의 데이터셋을 통해 Fine-tuning(미세 조정)을 수행할 수 있습니다.

2. 사전 학습된 모델 선택
한국어에 대한 다양한 사전 학습된 모델이 있습니다. 대표적으로 다음과 같은 모델들이 있습니다:

KoBERT: 한국어에 특화된 BERT 모델로, 법률 및 일반 텍스트 분석에 효과적입니다.
DistilBERT: BERT의 경량화 모델로, 속도가 빠르며 메모리 효율성이 뛰어납니다.
GPT-2: 자연어 생성에 강점을 가진 모델로, 다양한 태스크에 응용 가능합니다.

3. 데이터 전처리
경찰 범죄 분야의 법률 데이터셋을 준비합니다. 데이터는 사건의 설명, 법률 조항, 감정 레이블 등을 포함해야 합니다. 이를 위한 기본적인 전처리 단계는 다음과 같습니다:

텍스트 정제: HTML 태그, 특수 문자, 숫자 등을 제거합니다.
형태소 분석: 한국어의 어근을 추출하고 맥락을 이해하기 위해 형태소 분석기를 사용합니다. 예를 들어, KoNLPy 패키지를 사용할 수 있습니다.
감정 레이블링: 데이터에 긍정, 부정, 중립과 같은 감정 레이블을 부여합니다.
python

In [ ]:
import pandas as pd
from konlpy.tag import Okt

# 데이터 불러오기
data = pd.read_csv('crime_data.csv')

# 형태소 분석기 초기화
okt = Okt()

# 텍스트 정제 및 형태소 분석
data['cleaned_text'] = data['text'].apply(lambda x: ' '.join(okt.morphs(x)))


4. 모델 학습
사전 학습된 모델을 Fine-tuning하여 감정 분석을 수행합니다. 예를 들어, transformers 라이브러리를 사용할 수 있습니다.

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset

# 데이터셋 클래스 정의
class CrimeDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 모델 및 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=3)

# 데이터셋 생성
max_len = 128
dataset = CrimeDataset(data['cleaned_text'].tolist(), data['label'].tolist(), tokenizer, max_len)

# 학습 인자 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Trainer 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

# 모델 학습
trainer.train()


5. 감정 분석
학습된 모델을 사용하여 새로운 사건 텍스트에 대한 감정을 분석합니다.

In [ ]:
# 감정 예측 함수
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions.item()

# 새로운 텍스트에 대한 감정 예측
new_text = "사건의 내용에 대한 설명이 들어갑니다."
sentiment = predict_sentiment(new_text)
print("예측된 감정:", sentiment)  # 예: 0 - 부정, 1 - 중립, 2 - 긍정

6. 요약 및 고려사항
어근과 맥락: 형태소 분석을 통해 어근을 추출하고, 전이학습을 통해 맥락을 고려하여 감정 분석 모델을 학습합니다.
데이터 다양성: 경찰 범죄 분야의 다양한 사례와 사건 설명을 포함하여 모델의 일반화 능력을 향상시킬 수 있습니다.
사전 학습된 모델 활용: 한국어에 적합한 사전 학습된 모델을 사용하면 성능 향상에 도움이 됩니다.

=========================================================

1. 파인튜닝 (Fine-tuning) 추가
파인튜닝을 통해 사전 학습된 모델을 특정 태스크에 맞게 조정할 수 있습니다. 아래 코드는 KoBERT 모델을 사용하여 감정 분석을 위한 파인튜닝을 수행합니다.

1.1 감정 분석을 위한 데이터 전처리
기본적인 데이터 전처리를 수행하고, 학습 및 검증 데이터로 나눕니다.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from konlpy.tag import Okt

# 데이터 불러오기
data = pd.read_csv('crime_data.csv')

# 형태소 분석기 초기화
okt = Okt()

# 텍스트 정제 및 형태소 분석
data['cleaned_text'] = data['text'].apply(lambda x: ' '.join(okt.morphs(x)))

# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(data['cleaned_text'], data['label'], test_size=0.2, random_state=42)


1.2 데이터셋 클래스 정의
사전 학습된 모델을 입력으로 사용하기 위해 PyTorch의 Dataset 클래스를 정의합니다.

In [ ]:
from torch.utils.data import Dataset

class CrimeDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


1.3 모델 학습
파인튜닝을 위한 학습 설정을 수행합니다.

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# 모델 및 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=3)

# 데이터셋 생성
max_len = 128
train_dataset = CrimeDataset(X_train.tolist(), y_train.tolist(), tokenizer, max_len)
val_dataset = CrimeDataset(X_val.tolist(), y_val.tolist(), tokenizer, max_len)

# 학습 인자 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy='epoch',  # 에포크마다 평가
)

# Trainer 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# 모델 학습
trainer.train()


2. 문장 유사도 분석 추가
문장 유사도 분석을 위해 사전 학습된 BERT 모델을 사용하여 문장의 임베딩을 생성하고, 코사인 유사도를 계산합니다.

2.1 문장 임베딩 생성
문장을 임베딩으로 변환하는 함수를 정의합니다

In [ ]:
import torch
from sklearn.metrics.pairwise import cosine_similarity

def get_sentence_embedding(sentence, tokenizer, model):
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    # [CLS] 토큰의 출력을 문장 임베딩으로 사용
    return outputs.logits[0][0].numpy()  # 첫 번째 문장의 임베딩


2.2 유사도 계산
문장 유사도를 계산하는 함수를 정의합니다.

In [ ]:
def calculate_similarity(sentence1, sentence2, tokenizer, model):
    embedding1 = get_sentence_embedding(sentence1, tokenizer, model)
    embedding2 = get_sentence_embedding(sentence2, tokenizer, model)
    
    # 코사인 유사도 계산
    similarity = cosine_similarity([embedding1], [embedding2])
    return similarity[0][0]


3. 예제 사용법
이제 모델을 사용하여 감정 분석과 문장 유사도 분석을 수행할 수 있습니다.

3.1 감정 분석 예시

In [ ]:
# 감정 예측 함수
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions.item()

# 새로운 텍스트에 대한 감정 예측
new_text = "사건의 내용에 대한 설명이 들어갑니다."
sentiment = predict_sentiment(new_text)
print("예측된 감정:", sentiment)  # 예: 0 - 부정, 1 - 중립, 2 - 긍정


In [ ]:
3.2 문장 유사도 예시

# 문장 유사도 계산
sentence1 = "피고가 범죄를 저질렀습니다."
sentence2 = "피고는 범죄를 저질렀다는 주장이 있습니다."

similarity = calculate_similarity(sentence1, sentence2, tokenizer, model)
print("문장 유사도:", similarity)


In [ ]:
4. 요약
이 코드는 한국어 법률 분야에서 감정 분석과 문장 유사도를 평가하기 위한 파인튜닝된 모델을 사용한 예를 보여줍니다.

감정 분석: 법률 문서나 사건 설명의 감정을 분류합니다.
문장 유사도 분석: 사건의 다양한 문장 간의 유사도를 평가하여 유사한 사건을 식별하거나 사건의 중요성을 분석합니다.